In [1]:
#importing necessary libraries

#Beautiful Soup will help us read the Html document
from bs4 import BeautifulSoup as soup
#allows you to send HTTP requests
from requests import get
import requests
import re
import pandas as pd

In [2]:
#requesting data from url
response = requests.get("https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city")
response

<Response [200]>

In [3]:
response.text[:2000] #returns the first 2000 characters

'<!doctype html> <html lang="en"><head><meta http-equiv="Content-type" content="text/html; charset=utf-8"><title>Property for Sale in Hyderabad | 35981+ Hyderabad Properties for sale</title><meta name="description" content="Search 35981+ Properties for sale in Hyderabad on Makaan.com. Find &#10003;3908+ New Projects for sale. &#10003;13505+ Flats/Apartments. &#10003;4716+ Houses/Villas. Visit Now !"><meta name="keywords" content="Buy Hyderabad Properties, Residential Property for sale in Hyderabad, Property for sale in Hyderabad, Hyderabad Property Sale"><meta name="theme-color" content="#fff" id="themeColor"><meta content="origin" name="referrer"><meta name="p:domain_verify" content="55ce01b3ca93c05fd5a41439a23dd0d9"><meta name="fb:pages" content="155462194517712"><meta name="country" content="India"><meta name="og:type" content="website"><meta name="og:site_name" content="Makaan.com"><meta name="og:image:url" content="http://static.makaan.com/6/12/355/11020788.jpeg"><meta name="og:ti

In [4]:
data= soup(response.text)
house_containers = data.find_all("li",class_= "cardholder")
first = house_containers[0]
x = first.find_all('td', class_ = 'price')[0].text
x

' 1.03 Cr'

In [5]:
#creating empty list for appending the extracted value
title = []
location = []
price = []
price_per_sqft = []
area_in_sqft = []
building_status = []

In [6]:
%%time
n_pages = 0
for page in range(3,10): #the no. of pages to be extracted considering the pages available in the url
    n_pages +=1
    url = 'https://www.makaan.com/hyderabad-residential-property/buy-property-in-hyderabad-city?'+'&page='+str(page)
    
    # this for gets uns into next page after every iteration.
    r = requests.get(url) #here r is response data
    page_html = soup(r.text, 'html.parser')
    house_containers = page_html.find_all('li', class_ = 'cardholder')# class containing overall data of a property
    
    ##### title
    for data in house_containers:
        type_ = data.find_all("a", class_="typelink") 
        for i in type_:
            d = i.text
            title.append(d)
            
    ########Location 
        location_ = data.find_all("a", class_ = "loclink")
        for i in location_:
            d = i.text
            r= d.split(',',)[0]#splitting the obtained text and returning the first element of text
            location.append(r)
            
            
    ##########price
        cost = data.find_all("td", class_ = "price")
        for i in cost:
            d= i.text
            if 'L' in d:
                a = d.split()[0]
                price.append(float(a))
            elif 'Cr' in d:
                b = d.split()[0]
                price.append(float(b)*100)
            else:
                e = 0
                price.append(float(e))
                
    ####### price per sqft
    
        rate_sqft = data.find_all('td', class_="lbl rate")
        for i in rate_sqft:
            d = i.text
            res = d.split('/',)[0]
            r = re.sub(",","",res)
            price_per_sqft.append(r)
            
    ############################   AREA IN SQFT    ####################################

        area_sqft=data.find_all('td',class_="size")
        for i in area_sqft:
            d=i.text
            area_in_sqft.append(d)
            
##########################   BUILDING STATUS    ###################################

        building_status_=data.find_all("td",class_="val")
        for i in building_status_:
            d=i.text
            building_status.append(d)

###################################################################################

print('You scraped {} pages containing {} properties.'.format(n_pages, len(title)))
# Returns the number of pages scrapped and time taken for scarpping those pages.

You scraped 7 pages containing 140 properties.
Wall time: 15.3 s


In [7]:
df = pd.DataFrame()

In [8]:
df['title']=title 
df['location']=location
df['price(L)']=price
df['rate_persqft']=price_per_sqft
df['area_insqft']=area_in_sqft
df['building_status']=building_status

In [9]:
df.head()
df.shape

(140, 6)

In [10]:
df.to_csv('projectfinal.csv')

In [11]:
df=pd.read_csv('projectfinal.csv')

In [12]:
# Dropping of the unnecessary column "Unnamed: 0".

df.drop(columns='Unnamed: 0',axis=1,inplace=True)

In [13]:
df.describe()

,price(L),rate_persqft,area_insqft
count,140.000000,140.000000,140.000000
mean,85.060214,4142.335714,2060.192857
std,59.673168,1709.143416,1127.561250
min,6.000000,368.000000,200.000000
25%,45.797500,3399.750000,1403.000000
50%,74.790000,4497.500000,1726.500000
75%,107.250000,5100.000000,2462.500000
max,400.000000,8888.000000,9000.000000


In [14]:
# Drops the duplicate entires in the dataset.

df=df.drop_duplicates()

In [15]:

# As number of rows would vary we need to reset index.

df=df.reset_index()

In [16]:
# Dropping unnecessary columns in dataset.

df=df.drop(labels='index',axis=1)

In [17]:
df.shape

(140, 6)

In [18]:
df.head()

,title,location,price(L),rate_persqft,area_insqft,building_status
0,Residential Plot,Gachibowli,200.00,5555,3600,New
1,2 BHK Apartment,Bachupally,38.00,3368,1128,Ready to move
2,Residential Plot,Yadagirigutta,24.50,778,3150,New
3,2 BHK Apartment,Kukatpally,56.79,4960,1145,Ready to move
4,2 BHK Apartment,Raviryal,25.00,1785,1400,Under Construction


In [19]:
df.tail()

,title,location,price(L),rate_persqft,area_insqft,building_status
135,2 BHK Apartment,Adibatla,41.79,3803,1100,Under Construction
136,Residential Plot,Alair,18.39,511,3600,New
137,Residential Plot,Alair,9.19,511,1800,New
138,Residential Plot,Alair,7.35,511,1440,New
139,Residential Plot,Alair,13.79,511,2700,New


In [20]:
#group by and count the number of appertiment per location, average price, mode price, average rate per square ft.
#aggregating per title
agg_df = df.groupby('title').agg({'price(L)':['count','mean','max','min'],
                                 'rate_persqft':['mean','max','min'],
                                 'area_insqft':['mean','max','min']})

In [21]:
#agg_title_df = df.groupby('title').agg(average_price=('price(L)', 'mean'), average_area=('area_insqft', 'mean'),)

In [22]:
agg_df.columns = ["_".join(x) for x in agg_df.columns.ravel()]
agg_df= agg_df.reset_index()

In [23]:
agg_df.columns

Index(['title', 'price(L)_count', 'price(L)_mean', 'price(L)_max',
       'price(L)_min', 'rate_persqft_mean', 'rate_persqft_max',
       'rate_persqft_min', 'area_insqft_mean', 'area_insqft_max',
       'area_insqft_min'],
      dtype='object')

In [24]:
import seaborn as sns
import matplotlib.pylab as plt
plt.figure(figsize=(10,5))
#sns.set(style="whitegrid")
ax = sns.barplot(x="title", y="price(L)_mean", data=agg_df)
ax.set(xlabel='Average Price', ylabel='House Type')

plt.xticks(
    rotation=45, 
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'  
)

C:\Users\Gilbe\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), <a list of 9 Text xticklabel objects>)

In [25]:
#aggregating per location
agg_location_df = df.groupby('location').agg({'price(L)':['count','mean','max','min'],
                                 'rate_persqft':['mean','max','min'],
                                 'area_insqft':['mean','max','min']})

In [26]:
agg_location_df

price(L)                             rate_persqft  \
                             count        mean     max     min         mean   
location                                                                      
AS Rao Nagar                     1  120.000000  120.00  120.00  4800.000000   
Adibatla                        12   45.422500   75.00   18.62  3366.416667   
Alair                           21   76.242857  183.00    7.35  2849.380952   
Almasguda                        1  125.000000  125.00  125.00  5208.000000   
Amangal                          1   10.000000   10.00   10.00   556.000000   
Appa Junction Peerancheru        1  235.000000  235.00  235.00  7200.000000   
Aushapur                         1   45.000000   45.00   45.00  5000.000000   
Bachupally                       6   66.120000   87.63   38.00  4278.000000   
Bandlaguda Jagir                 5   77.604000   90.22   43.00  4941.800000   
Beeramguda                       1   88.000000   88.00   88.00  4386.000000   
Bhanur                           1   69.610000   69.61   69.61  2900.000000   
Bhuvanagiri                      1   10.000000   10.00   10.00  5000.000000   
Boduppal                         1   85.000000   85.00   85.00  4473.000000   
Chandanagar                      2   59.095000   67.39   50.80  4900.000000   
Gachibowli                       2  300.000000  400.00  200.00  5555.000000   
Gandhinagar                      1    8.000000    8.00    8.00   604.000000   
Gopanpally                       5  108.526000  121.00   96.62  5600.600000   
Himayat Nagar                    1  240.000000  240.00  240.00  8888.000000   
Ibrahimpatnam                    1    6.000000    6.00    6.00   368.000000   
Kismatpur                        6   78.755000   89.13   70.38  4666.666667   
Kokapet                          2  184.000000  244.00  124.00  6795.000000   
Kompally                         2  219.500000  234.00  205.00  7705.500000   
Kondapur                         2  108.480000  130.00   86.96  6247.000000   
Kukatpally                       1   56.790000   56.79   56.79  4960.000000   
Kushaiguda                       1   43.040000   43.04   43.04  4000.000000   
Maheshwaram                      5   89.382000  201.00   15.20  3039.400000   
Mallampet                        2   83.000000   90.00   76.00  4326.000000   
Manikonda                        1   88.350000   88.35   88.35  5000.000000   
Miyapur                          1   79.200000   79.20   79.20  5126.000000   
Mokila                           6   49.418333   61.75   44.84  3833.333333   
Nacharam                         1   47.000000   47.00   47.00  4476.000000   
Nagole                           1   85.890000   85.89   85.89  5250.000000   
Nallagandla Gachibowli           1  120.000000  120.00  120.00  6499.000000   
Nanakramguda                     1   76.000000   76.00   76.00  6333.000000   
Narsingi                         2  107.000000  108.00  106.00  6290.000000   
Nizampet                        13   70.404615   89.25   52.42  4707.923077   
Patancheru                       1   41.580000   41.58   41.58  2444.000000   
Pati                             1  166.000000  166.00  166.00  6384.000000   
Pocharam Near Muthangi           1   42.200000   42.20   42.20  2445.000000   
Puppalaguda                      1   75.000000   75.00   75.00  6000.000000   
Raviryal                         1   25.000000   25.00   25.00  1785.000000   
Sadashivpet                      2   16.035000   19.34   12.73  1011.500000   
Sanath Nagar                     1  140.000000  140.00  140.00  7446.000000   
Sangareddy                       1   11.820000   11.82   11.82   722.000000   
Shadnagar                        1   16.200000   16.20   16.20   640.000000   
Shamirpet                        1   11.400000   11.40   11.40  1055.000000   
Shankarpalli                     1   60.000000   60.00   60.00   666.000000   
Shankarpally Road                1   85.000000   85.00   85.00  5666.000000 